# Creating the Channel Funding Transaction

In this section, we’ll create a lightning channel funding transaction from scratch using Python. We’ll walk through each part of the transaction—how it’s constructed, signed, and how peers exchange messages to share the necessary information to make it happen. We'll test everything using Bitcoin Core in regtest mode.

## Setup

For this notebook, we’ll use Test Framwork the created on `chapter 0 - lightning node keys derivation`.

In [1]:
%run "../chapter 0 - lightning node keys derivation/lightning node keys derivation.ipynb"

2025-11-12T15:05:29.418000Z TestFramework (INFO): PRNG seed is: 4405791796561775034
2025-11-12T15:05:29.420000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_vyc2ovz5
🟢 New TestShell started. Block height: 0
Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Payment BasePoint:  025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Bob Payment BasePoint:  02f98efd3f2b2fbe7bd83c419f5f64f8280798b8a9175fdb77c0091bbb95c79506
To obscure commitment number 0xb433fd43a66f


In [2]:
from functions.test_framework.key import generate_bip340_key_pair
from functions.test_framework.address import program_to_witness
from functions.test_framework.script import tagged_hash
from functions.bip0327.reference import individual_pk, key_agg, get_xonly_pk, key_sort, apply_tweak, nonce_gen, nonce_agg, SessionContext, sign, partial_sig_verify, partial_sig_agg, schnorr_verify

### Setup Alice and Bob Funding Outputs

Create and fund Taproot addresses for Alice and Bob so they have funds to open a simple Taproot channel. The UTXO created for Alice will be the input of the Channel Funding Transaction.

In [3]:
# Create a new Alice key pair for channel funding multisig output
alice_funding_privkey, alice_funding_pubkey = generate_bip340_key_pair()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_funding_address = program_to_witness(0x01, alice_funding_pubkey.get_bytes())
print(f"Alice funding pubkey: {alice_funding_pubkey.get_bytes().hex()}")
print(f"Alice funding privkey: {alice_funding_privkey.get_bytes().hex()}")
print(f"Alice funding address: {alice_funding_address}")

# Create a new Bob key pair for channel funding multisig output
bob_funding_privkey, bob_funding_pubkey = generate_bip340_key_pair()
# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
bob_funding_address = program_to_witness(0x01, bob_funding_pubkey.get_bytes())
print(f"Bob funding pubkey: {bob_funding_pubkey.get_bytes().hex()}")
print(f"Bob funding privkey: {bob_funding_privkey.get_bytes().hex()}")
print(f"Bob funding address: {bob_funding_address}")

# Send 1 BTC to Alice and Bob
alice_initial_fund = 1
bob_initial_fund = 1
txid_to_spend = node.sendmany("", {alice_funding_address: alice_initial_fund, bob_funding_address: bob_initial_fund})
print(f"Transaction creating Alice UTXO: {txid_to_spend}")
rawtx_json = node.getrawtransaction(txid_to_spend, True)
print(json.dumps(rawtx_json, indent=2, default=str))

expected_spk = (b"\x51" + b"\x20" + alice_funding_pubkey.get_bytes()).hex()
# Get the index of the Alice output to spend
index_to_spend = next(
    v["n"] for v in rawtx_json["vout"]
    if v["scriptPubKey"]["hex"] == expected_spk
)
print(f"Alice transaction to spend: {txid_to_spend}:{index_to_spend}")

result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

Alice funding pubkey: b425dfd9363c40bb9c8dda76f7bf5de038f05bd8755402fa073d7bf4c722aa0f
Alice funding privkey: bed58c7fda57f4daa7c0ebf5f07e888be43effa8e75b368c6a38d577a482c5dd
Alice funding address: bcrt1pksjalkfk83qth8ydmfm0006auqu0qk7cw42q97s884alf3ez4g8sqq2ha2
Bob funding pubkey: dc09d267b64e16c3371fb880eac0b6b4926b008992403f09dd4a3e1bcfa8463c
Bob funding privkey: 6de4627887e7ee044d8d67f220bb2e1e8717bf2879de0f87cb708706ccad4e1d
Bob funding address: bcrt1pmsyayeakfctvxdclhzqw4s9kkjfxkqyfjfqr7zwafglphnaggc7qr2k807
Transaction creating Alice UTXO: af1379ec8d047f4ba7c84dfd58b8e84941e13d55c24121a692cff3faca60ebad
{
  "txid": "af1379ec8d047f4ba7c84dfd58b8e84941e13d55c24121a692cff3faca60ebad",
  "hash": "56d83af36db1b3e1acedc341c582c63e9579f2d1530438aac4cc63d0c97ce463",
  "version": 2,
  "size": 289,
  "vsize": 208,
  "weight": 829,
  "locktime": 101,
  "vin": [
    {
      "txid": "e7f6fd542213524c27772fb9f4685cd8abf1204a0e2a56daf1dbb87ecf04a0ae",
      "vout": 0,
      "scriptSig": {
    

## Channel Establishment

The "[Extention Bolt](https://github.com/lightning/bolts/pull/995)" for simple taproot channels (work in progress) defines a pathway to create a channel. The messages defined in [BOLT 2](https://github.com/lightning/bolts/blob/master/02-peer-protocol.md#channel-establishment-v1) are used to exchange the information needed to create the Channel Funding Transaction, as shown below.


    +-----------+                                                            +---------+
    |           |--(1)---------- open_channel (next local nonce) ----------->|         |
    |           |<-(2)---------- accept_channel (next local nonce) ----------|         |
    |           |                                                            |         |
    |   Alice   |--(3)------  funding_created (partial sig) (jit nonce)----->|   Bob   |
    |           |<-(4)------  funding_signed (partial sig) (jit nonce)-------|         |
    |           |                                                            |         |
    |           |--(5)---------- channel_ready (next local nonce) ---------->|         |
    |           |<-(6)---------- channel_ready (next local nonce) -----------|         |
    +-----------+                                                            +---------+


* (1) open_channel - Alice share its "next_local_nonce"

* (2) accept_channel - Bob share its "next_local_nonce"

* (3) funding_created - Alice samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (4) funding_signed - Bob samples a fresh nonce, combines it with next_local_nonce received to create a MuSig2 partial signature, then shares partial_signature_with_nonce.

* (5) channel_ready - Alice share its "next_local_nonce"

* (6) channel_ready - Bob share its "next_local_nonce"


All the nonces are generated using "NonceGen" algorithm defined in [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki) to ensure it generates nonces in a safe manner.

The pubkeys are sorted using the "KeySort" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To compute the aggregated musig2 public key from the sorted funding_pubkeys use the "KeyAgg" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

To construct a musig2 partial signature for the sender's remote commitment use the "Sign" algorithm from [bip-musig2](https://github.com/bitcoin/bips/blob/master/bip-0327.mediawiki).

### The Channel Funding Transaction

Alice has a 1 BTC UTXO that will serve as the funding transaction input. We’ll now create two outputs:

#### The change output back to Alice

In [4]:

# Create a new Alice key pair output for the change
alice_change_privkey, alice_change_pubkey = generate_bip340_key_pair()

# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
alice_change_address = program_to_witness(0x01, alice_change_pubkey.get_bytes())
print(f"Alice funding pubkey: {alice_change_pubkey.get_bytes().hex()}")
print(f"Alice funding privkey: {alice_change_privkey.get_bytes().hex()}")
print(f"Alice funding address: {alice_change_address}")


Alice funding pubkey: 616d6089b74fb7c92d81498f31a0faf4ce5c5b024b0b222b293bd58f2312f47e
Alice funding privkey: 3d6603ab55016406ee21de8c6df24c63df7f2c108065895fa9f7de963ff26cd2
Alice funding address: bcrt1pv9kkpzdhf7mujtvpfx8nrg867n89ckczfv9jy2ef802c7gcj73lqqwc4wv


#### The channel musig funding output

The diagram below illustrates how a Taproot channel funding output is constructed. [BIP 341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki#cite_ref-22-0) states: "If the spending conditions do not require a script path, the output key should commit to an unspendable script path instead of having no script path. This can be achieved by computing the output key point as Q = P + int(hashTapTweak(bytes(P)))G. And [BIP 86](https://github.com/bitcoin/bips/blob/master/bip-0086.mediawiki) define the following: 

```
internal_key:       lift_x(derived_key)
32_byte_output_key: internal_key + int(HashTapTweak(bytes(internal_key)))G
```

As our internal key is the agregated public key of Alice and Bob, we have:


    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^
                   |   +----------+
                    ---| Pagg + T |
                       +----------+
                         ^      ^
                         |      |  +-----------+
                         |       --| T = t * G |<---------
                         |         +-----------+          |
                         |                                |
           +---------------------------------------+    +---+   +-------------------------------+
           |      generate_musig_key(Pa, Pb)       |    | t | = | TaggedHash ("Taptweak", Pagg) |
           +---------------------------------------+    +---+   +-------------------------------+

In [5]:
# Create a new Alice key pair for channel funding musig2 output
privkey_alice_musig2, pubkey_alice_musig2 = generate_bip340_key_pair()
print(f"Alice MuSig2 pubkey: {pubkey_alice_musig2.get_bytes().hex()}")
print(f"Alice MuSig2 privkey: {privkey_alice_musig2.get_bytes().hex()}")

# Create a new Bob key pair for channel funding musig2 output
privkey_bob_musig2, pubkey_bob_musig2 = generate_bip340_key_pair()
print(f"Bob MuSig2 pubkey: {pubkey_bob_musig2.get_bytes().hex()}")
print(f"Bob MuSig2 privkey: {privkey_bob_musig2.get_bytes().hex()}")

# Generate a 2-of-2 tweaked aggregate MuSig2 key using the pubkeys form Alice and Bob
sorted_pubkeys = key_sort([pubkey_alice_musig2.get_bytes(bip340=False), pubkey_bob_musig2.get_bytes(bip340=False)])
key_agg_ctx = key_agg(sorted_pubkeys)
agg_pubkey = get_xonly_pk(key_agg_ctx)

# Compute the BIP86 tweak
bip86_tweak = tagged_hash("TapTweak", agg_pubkey)
tweaked_ctx = apply_tweak(key_agg_ctx, bip86_tweak, is_xonly=True)

# Final Taproot output key (x-only) to put in the P2TR scriptPubKey
agg_pubkey_tweaked = get_xonly_pk(tweaked_ctx)

print(f"Aggregate musig2 pubkey: {agg_pubkey_tweaked.hex()}")

# Derive the bech32 address
# Use program_to_witness(version_int, pubkey_bytes)
channel_funding_musig_address = program_to_witness(0x01, agg_pubkey_tweaked)
print(f"Channel funding musig2 output address: {channel_funding_musig_address}")

Alice MuSig2 pubkey: 17f7a4ea83861733641ed5093d4e4152a75adde65910253c299ea4f01cbd0819
Alice MuSig2 privkey: 9021ccdd929c021885ada30ad2cf0f1e3ccd7fa62c85d0f1db9c078029cbed64
Bob MuSig2 pubkey: 1a68f5ff9613f00224f226df8f53984875c4ba133ebb2f0d9010d4f5c912e2e5
Bob MuSig2 privkey: d4335ddaf839b8b7cabe29ebcc06c43f1400ce62a9b0f3e99b972cbd5744ba5b
Aggregate musig2 pubkey: 474f599ec188027db691c784fd6cce810fc515f5bdd18fd302abc0d89f794e2e
Channel funding musig2 output address: bcrt1pga84n8kp3qp8md53c7z06mxwsy8u2904hhgcl5cz40qd38mefchq8n2svt


#### The channel funding transaction

##### The unsigned transaction

In [6]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(txid_to_spend))[::-1]
index = index_to_spend.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("02")

# Transaction Fee
tx_fee_sat = int(float("0.00000300") * 100000000)

# OUTPUT 1 - Channel Funding Output
# 0.01 BTC to the 2-of-2 multisig Taproot output
channel_value_sat = int(float("0.01") * 100000000)
channel_value = channel_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
channel_spk = bytes.fromhex("51") + varint_len(agg_pubkey_tweaked) + agg_pubkey_tweaked

# OUTPUT 2 - Change Output
# The rest back to Alice's change address
# 1 BTC - 0.01 BTC - 0.0001 BTC = 0.9899 BTC
alice_change_value_sat = int(float(alice_initial_fund) * 100000000) - channel_value_sat - tx_fee_sat
alice_change_value = alice_change_value_sat.to_bytes(8, byteorder="little", signed=False)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
alice_change_spk = bytes.fromhex("51") + varint_len(alice_change_pubkey.get_bytes()) + alice_change_pubkey.get_bytes()

outputs = (
    channel_value
    + varint_len(channel_spk)
    + channel_spk
    + alice_change_value
    + varint_len(alice_change_spk)
    + alice_change_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  0200000001adeb60cafaf3cf92a62141c2553de14149e8b858fd4dc8a74b7f048dec7913af0000000000ffffffff0240420f0000000000225120474f599ec188027db691c784fd6cce810fc515f5bdd18fd302abc0d89f794e2e949de60500000000225120616d6089b74fb7c92d81498f31a0faf4ce5c5b024b0b222b293bd58f2312f47e00000000
{
  "txid": "6d1668e69e33d0b6c925c33e4089a6478076f129667446b77c6a9ba2cbe39aa5",
  "hash": "6d1668e69e33d0b6c925c33e4089a6478076f129667446b77c6a9ba2cbe39aa5",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 0,
  "vin": [
    {
      "txid": "af1379ec8d047f4ba7c84dfd58b8e84941e13d55c24121a692cff3faca60ebad",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": "0.01000000",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 474f599ec188027db691c784fd6cce810fc515f5bdd18fd302abc0d89f794e2e",
        "desc": "rawtr(474f599ec188027db691c784fd6cce810fc515f5bdd18fd302abc0d8

##### The sighash for the key path spend

This is the “Schnorr key spend” case: you’re proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [7]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index).digest()

input_amount_sat = int(alice_initial_fund * 100_000_000)
input_amounts = input_amount_sat.to_bytes(8, byteorder="little", signed=False)
sha_amounts = sha256(input_amounts).digest()

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
spk = bytes.fromhex("51") + varint_len(alice_funding_pubkey.get_bytes()) + alice_funding_pubkey.get_bytes()

sha_scriptpubkeys = sha256(
    varint_len(spk)
    + spk
).digest()

sha_sequences = sha256(sequence).digest()

sha_outputs = sha256(outputs).digest()

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode()).digest()
msg = tag_hash + tag_hash + sig_msg
sighash = sha256(msg).digest()


##### Signing the sighash

In [8]:
signature = alice_funding_privkey.sign_schnorr(sighash) 

##### The signed transaction

Finally the last step is to add the signature to the witness field and she has the signed transaction!

In [9]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(signature)
    + signature
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

funding_channel_txid = node.sendrawtransaction(signed_tx.hex())
result = node.generatetoaddress(nblocks=1, address=address, called_by_framework=True)

print("funding_channel_txid: ", funding_channel_txid)

signed tx:  02000000000101adeb60cafaf3cf92a62141c2553de14149e8b858fd4dc8a74b7f048dec7913af0000000000ffffffff0240420f0000000000225120474f599ec188027db691c784fd6cce810fc515f5bdd18fd302abc0d89f794e2e949de60500000000225120616d6089b74fb7c92d81498f31a0faf4ce5c5b024b0b222b293bd58f2312f47e0140f030ec6f630df6dbe37c6efbb4c3cc8f0f21eae27b937b05f8e21467a7bc159dd5c988d07e8bb486a92f51c67deeeee8ea20d16ee4988706d179aeb242f8c4a700000000
{
  "txid": "6d1668e69e33d0b6c925c33e4089a6478076f129667446b77c6a9ba2cbe39aa5",
  "hash": "f11f99e7b8f8e7c599714df4f037082a4fb2eb9f443f81ae18be98c1ee8d27f3",
  "version": 2,
  "size": 205,
  "vsize": 154,
  "weight": 616,
  "locktime": 0,
  "vin": [
    {
      "txid": "af1379ec8d047f4ba7c84dfd58b8e84941e13d55c24121a692cff3faca60ebad",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "f030ec6f630df6dbe37c6efbb4c3cc8f0f21eae27b937b05f8e21467a7bc159dd5c988d07e8bb486a92f51c67deeeee8ea20d16ee4988706d179aeb242f